In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open("names.txt").read().splitlines()

In [5]:
len(words)

32033

In [17]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [33]:
block_size = 3
X, y = [], []


for w in words[:5]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        #print(ix," -> ",ch)
        X.append(context)
        y.append(ix)
        print(''.join(itos[i] for i in context), "--->", itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
y = torch.tensor(y)

emma
... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
olivia
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
ava
... ---> a
..a ---> v
.av ---> a
ava ---> .
isabella
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
sophia
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [34]:
X.shape, y.shape, X.dtype, y.dtype

(torch.Size([32, 3]), torch.Size([32]), torch.int64, torch.int64)

In [35]:
C = torch.randn((27,2))

In [56]:
C[X].shape
X[13,2]
C[X][13,2]
C[1]

tensor([-1.0778, -0.4870])

In [57]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [58]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [71]:
torch.cat([emb[:,0,:], emb[:,1,:], emb[:,2,:]],1).shape
torch.cat(torch.unbind(emb, 1), 1).shape
emb.view(32,6).shape

torch.Size([32, 6])

In [75]:
h = torch.tanh(emb.view(-1,6) @ W1 + b1)

In [77]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [79]:
logits = h @ W2 + b2

In [83]:
counts = logits.exp()
probs = counts / counts.sum(1, keepdim = True)

In [84]:
probs[0].sum()

tensor(1.0000)

In [86]:
loss = -probs[torch.arange(y.shape[0]), y].log().mean()
loss

tensor(14.5507)

### One Go MLP

In [109]:
block_size = 3
X, y = [], []


for w in words:
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        y.append(ix)
        context = context[1:] + [ix]

X = torch.tensor(X)
y = torch.tensor(y)

In [110]:
X.shape, y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [111]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [112]:
sum(p.nelement() for p in parameters)

3481

In [113]:
for p in parameters:
    p.requires_grad = True

In [130]:
for k in range(1000):

    #minibatch
    ix = torch.randint(0,X.shape[0], (32,))
    #forwrd
    emb = C[X[ix]]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y[ix])

    #backward pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #update
    for p in parameters:
        p.data += -0.1 * p.grad

print(f"loss on minibatch= {loss.item()}")

loss on minibatch= 2.365237236022949


In [131]:
    emb = C[X]
    h = torch.tanh(emb.view(-1,6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(f"loss on full training set= {loss.item()}")

loss on full training set= 2.5476291179656982
